# GeoNet FDSN webservice with Obspy - GeoNet FDSN Clients

In this tutorial we will look at how to get both near real-time and archive waveform data. 

## Import Modules and Define Clients

In [1]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

arc_client  = 'https://service.geonet.org.nz'
# or arc_client = "GEONET"
nrt_client = 'https://service-nrt.geonet.org.nz'

## Option 1: By trial and error

You will need to specify the correct wave server to get the data you want. If you try to get data from one server and that raises an exception (because the data you requested are not available) repeat the request with the other server.

In [2]:
#try with "old" data that should be successful with archive client
t = UTCDateTime('2017-10-04')

#first try to get data from archive server
try:
    client = Client(arc_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('arc client successful')
#if this raises an exception, try the near real-time server
except:
    client = Client(nrt_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('nrt client successful')
print(st)

arc client successful
1 Trace(s) in Stream:
NZ.KRVZ.10.EHZ | 2017-10-04T00:00:00.003134Z - 2017-10-04T00:05:00.003134Z | 100.0 Hz, 30001 samples


In [3]:
#try with "recent" data that should be successful with near real-time client
#CAUTION, if the station is currently not operating, then the near real-time client will fail
t = UTCDateTime.now() - 86400 #1 day ago

#first try to get data from archive server
try:
    client = Client(arc_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('arc client successful')
#if this raises an exception, try the near real-time server
except:
    client = Client(nrt_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('nrt client successful')
print(st)

nrt client successful
1 Trace(s) in Stream:
NZ.KRVZ.10.EHZ | 2023-03-08T01:32:19.863127Z - 2023-03-08T01:37:19.863127Z | 100.0 Hz, 30001 samples


## Option 2: Use the time range to select client
Use the time range to select a client, if the time range requires both clients then use Option 3. 

In [4]:
#try with "recent" data that should be successful with near real-time client
#CAUTION, if the station is currently not operating, then the near real-time client will fail
starttime = UTCDateTime.now() - 518400 #6 days ago
endtime = starttime+300

days7 = UTCDateTime.now() - 604800 #7 days ago
days8 = UTCDateTime.now() - 691200 #8 days ago

if endtime < days7:
    client = Client(arc_client)
    print("Client is archive client")
elif starttime > days8:
    client = Client(nrt_client)
    print("Client is near real-time client")
else:
    print("Time range requires both clients")
    
st = client.get_waveforms('NZ', 'NEZ', '30', 'HDF', starttime, endtime)
print(st)

Client is near real-time client
1 Trace(s) in Stream:
NZ.NEZ.30.HDF | 2023-03-03T01:32:20.753130Z - 2023-03-03T01:37:20.753130Z | 100.0 Hz, 30001 samples


## Option 3: Request from both clients and merge
This is useful if the time window for the data request spans both time periods.

First, request data from both clients

In [5]:
#Define time period
t1 = UTCDateTime.now()-777600 #9 days ago
t2 = UTCDateTime.now()-518400 #6 days ago

#nrt client
try:
    client = Client(nrt_client)
    stnrt = client.get_waveforms('NZ', 'NAPT', '40', 'BTZ', t1, t2)
    print('nrt client successful')
except:
    print('nrt client not successful')

#arc client
try:
    client = Client(arc_client)
    starc = client.get_waveforms('NZ', 'NAPT', '40', 'BTZ', t1, t2)
    print('arc client successful')
except:
    print('arc client not successful')
print(stnrt,starc)

nrt client successful
arc client successful
9 Trace(s) in Stream:
NZ.NAPT.40.BTZ | 2023-03-01T01:31:59.500000Z - 2023-03-01T20:28:59.400000Z | 10.0 Hz, 682200 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:37:39.300000Z - 2023-03-01T20:38:19.800000Z | 10.0 Hz, 406 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:38:32.700000Z - 2023-03-01T20:43:45.500000Z | 10.0 Hz, 3129 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:44:33.800000Z - 2023-03-01T20:45:25.300000Z | 10.0 Hz, 516 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:43:45.600000Z - 2023-03-01T20:44:33.700000Z | 10.0 Hz, 482 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:45:25.400000Z - 2023-03-01T20:47:45.700000Z | 10.0 Hz, 1404 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:48:17.400000Z - 2023-03-01T20:48:44.300000Z | 10.0 Hz, 270 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:47:45.800000Z - 2023-03-01T20:48:17.300000Z | 10.0 Hz, 316 samples
NZ.NAPT.40.BTZ | 2023-03-01T20:48:44.400000Z - 2023-03-03T01:32:21.300000Z | 10.0 Hz, 1034170 samples 3 Trace(s) in Stream:
NZ.NAPT.40.BTZ | 2023

Now merge waveforms into a single stream object. 

In [6]:
st = stnrt
st += starc
st.merge(fill_value = 'interpolate')
print(st)

1 Trace(s) in Stream:
NZ.NAPT.40.BTZ | 2023-02-28T01:32:21.300000Z - 2023-03-03T01:32:21.300000Z | 10.0 Hz, 2592001 samples
